In [1]:
import pandas as pd
import numpy as np
import math
import os
#from IPython.display import display

# Made by Henric Pietro Vicente Gil please credit me in your work :) 
# henricgil@discente.ufg.br

In [3]:
"""
# This cell defines the location of the files, 

# path_to_files may be a folder where multiple files are;

# file_list is a list of all the files in the folder provided in path_to_files;

# s_file represents the file(s) to be read, can be either a list or a string

# uncomment the part that you will use, comment the part you wont
"""

# Points to a file ./myfolder/example.xlsx or to a folder c:\\my\\folder\\where\\my\\\files\\are
path_to_files = "D:\\The\\Directory\\where\\my\\file(s)\\is(are)"
try:
    os.mkdir("./results")
except FileExistsError:
    pass
#### To get from a directory

# file_list=os.listdir(path_to_files)
# s_file = [f"{path_to_files}\\{file}" for file in file_list]

#### To get from a path

s_file=path_to_files

FileExistsError: [WinError 183] Não é possível criar um arquivo já existente: 'results'

In [9]:
"""
# Defines the necessary variables used all through out the code

# duplicate_identifier_column is the collumn that identifies the duplicates in your code

# values_col defines the column where your dose/lc50/ic50... values are

# max_z_score is the z value used to separate the outliers from non outliers

# convert_to_p despite the name, this bool conditions the result into a -log(result) or pResult

# convert_measure bool that conditions if you need to convert measurements, also eliminates impossible values
"""
duplicate_identifier_column = "SMILES"
values_col = "LC50"
max_z_score = 1.8
convert_to_p = False # p = -log(measure)
convert_measure = False

##### DO NOT RUN UNLESS YOUR DATA CONTAINS NULL CHARS ########


In [ ]:
######### DO NOT RUN UNLESS YOUR DATA CONTAINS NULL CHARS ################

for file in s_file:        
        with open(file,"r") as f:
                a=f.read()

        a=a.replace("\x00","")
        
        with open(file,"w") as f:
                f.write(a)
        # df=pd.read_csv(file,delimiter=",")
        # print(df.head(1))

##### DONT RUN UNLESS YOUR FILES HAVE SOME COMPATIBILITY ISSUES #######


In [41]:
###### DONT RUN UNLESS YOUR FILES HAVE SOME COMPATIBILITY ISSUES #######
if type(s_file) is not str:
    for file in s_file:
        name=str(file).replace(path_to_files,"")
        name=name.replace(".csv",".xlsx")
        
        df=pd.read_csv(file,delimiter=",")
        df.to_excel(f"temp_dataset/{name}")

path_to_files = "./temp_dataset"
file_list=os.listdir(path_to_files)
s_file = [f"{path_to_files}\\{file}" for file in file_list]

### Defintively run

In [10]:
def main(files):
    
    def isInt(i):
        """Check if it is an integer"""
        try:
            int(i)
            return True
        except ValueError:
            return False

    def isFloat(f):
        try:
            float(f)
            return True
        except ValueError:
            return False

    def convertTo_mg(df, column=values_col):
        """
        Pass a pandas 'series like' (aka: df.column) of measurement strings, 
        convert the values into miligram numbers, 
        returns a list of lists with a number and the unit
        
        """

        micro = "\u00B5"
        measure = []

        for dose in df.loc[:, column]:
            # num=[n for n in number if isInt(n) or n=="."]
            num = ""
            unit = ""
            nums = []
            avg = 0
            # Ensures it is not nan or None
            if type(dose) is str:
                for i, n in enumerate(dose):
                    # Avoids unprecise entries
                    if n in ["<", ">"]:
                        continue

                    if isInt(n) or n == "." and num.find(".") == -1:
                        num += n

                    if isInt(n) == False:
                        unit += n
                        num = ""
                    if n in [",", "-", "\\", ".", "/"] and num.find(".") != 0 and num != "":
                        num.strip(" ,-\\/")
                        nums.append(float(num))
                        num = ""
                    if len(nums) > 1 and "" not in nums:
                        avg = np.mean(nums)

                    elif len(nums) == 0 and num != "":
                        avg = float(num)
                # unit=[str(s) for s in number if isInt(s)==False]
                measure.append([avg, str(unit).strip(" -.")])

            elif type(dose) is float or type(dose) is int:
                measure.append([dose, "mg"])
        for i, m in enumerate(measure):
            # print(m)
            u = str(m[1])
            n = m[0]

            # if you want to use the strings that contain unit values uncomment the commented lines
            if u.find("kg") != -1 and u.find("mg") == -1:
                measure[i][0] = n*1_000_000
                # measure[i][1]=str(measure[i][1]).replace("kg","mg")

            # might be with special char
            elif u.find(f"{micro}g") != -1 or unit.find("ug") != -1:
                measure[i][0] = n/1000
                # measure[i][1]=str(measure[i][1]).replace("ug","mg")

            elif u.find("cg") != -1:
                measure[i][0] = n*10
                # measure[i][1]=str(measure[i][1]).replace("cg","mg")
            if u.find("mg") == 0:
                measure[i][0] = n

            if u.find("g") == 0:
                measure[i][0] = n*1000
                # measure[i][1]=str(measure[i][1]).replace("g","mg")

        solved_col = pd.Series(measure)

        return solved_col
    
    # Method that calculates the standard deviation with a number list and the mean, but use np.std() instead
    def stdCalculation(numList, mean):
        n = len(numList)+1
        soma = 0
        for x in numList:
            soma += (x-mean)**2
        std = math.sqrt((soma)/n)
        return std

    # Method to calculate the z score for each number in a array of numbers, returns a list of z-scores of each number relevant to the array
    def z_scorer(nums: list, mean: float or int, std: float or int):

        z_scores = []
        for x in nums:
            z = (x-mean)/std
            z_scores.append(z)
        return z_scores

    # Most of the action happens here
    def idOutliers(df: pd.DataFrame, name_col: str = "Drug", value_col: str = "Dose", max_z: float = 1.8, convert_to_p: bool=True):
        dict_rows = {}
        means = {}
        # Comment if you have empty values
        df = df.dropna(axis=0, how="all")
        
        if convert_measure:
            values = [num for num, mg in df.loc[:, value_col]]
        else:
            values = [num for num in df.loc[:, value_col]]
        
        names = [drugs for drugs in df.loc[:, name_col] if type(drugs) is str]
        for i, name in enumerate(names):
            if name=="" or name=="-":
                name="Empty"
                
            if name not in dict_rows:
                dict_rows[name] = []
                
                dict_rows[name].append(values[i])
            else:
                dict_rows[name].append(values[i])

        for n in dict_rows:
            media = []
            if len(dict_rows[n]) > 1 and type(dict_rows[n]) is float:
                means[n] = np.mean(dict_rows[n])

                if dict_rows[n][0]!=0 and means[n]/dict_rows[n][0] != 1:
                    std = np.std(dict_rows[n])
                else:
                    std = 1

                z = z_scorer(dict_rows[n], means[n], std)

                for _i, i in enumerate(z):
                    if abs(i) <= max_z:
                        media.append(dict_rows[n][_i])

                med = np.mean(media)
                # atrubutes where in the column specified is equal to the name of the current row, and replaces the value (dose) of that row
                if convert_to_p:
                    df.loc[df[name_col] == n, value_col] = -math.log10(med)
                else:
                    df.loc[df[name_col] == n, value_col] = med
            else:
                if convert_to_p:
                    if dict_rows[n][0] <= 0:
                        df.loc[df[name_col] == n, value_col] = dict_rows[n][0]
                    else: 
                        df.loc[df[name_col] == n, value_col] = -math.log10(dict_rows[n][0])
                else:
                    df.loc[df[name_col] == n, value_col] = dict_rows[n][0]

        return df
    # computes in the order necessary to generate the dataframes

    def organize(file,*sheet):
        # read
        print(file)
        name=str(file).replace(path_to_files,"")
        name=name.replace(".xlsx","")
        
        if file.find(".csv")!=-1:
            df = pd.read_csv(file,delimiter=",")
            name=str(file).replace(path_to_files,"")

        elif file.find(".xlsx")!=-1 and sheet is not None:
            name=sheet
        df = pd.read_excel(file, sheet_name=sheet) 
            

        # convert?
        if convert_measure:
            new_doses = convertTo_mg(df, values_col)
            df[values_col] = new_doses
        # generate
        z = idOutliers(df, duplicate_identifier_column, values_col, max_z_score, convert_to_p)
        z = z.drop_duplicates(keep="first",subset=duplicate_identifier_column)
        
        
        z.to_excel(f"./results/{sheet}.xlsx")
        print(f"Generated: {sheet}.xlsx")
        # confirm
        if sheet=="Planilha1":
            print(f"generated {sheet}.xlsx dataframe")

    # Made to work with a single file and a list of files
    def read_through_files(files):

        if type(files) is str:
            if files.find(".csv")!=-1:
                organize(files)
            else:
                sheets = pd.ExcelFile(files).sheet_names
                for sheet in sheets:
                    if sheet != ".":
                        organize(files,sheet=sheet)
        else:
            for file in files:
                if file.find(".csv")!=-1:
                    organize(file)
                else:   
                    sheets = pd.ExcelFile(file).sheet_names
                    for sheet in sheets:
                        organize(file,sheet)

    read_through_files(files=files)

### Run to execute
###### Equivalent to if __name__ == __main__:

In [ ]:
# Execute everything
main(s_file)
    